# Movies Imformation - Retrieval  

## Librarys

In [1]:
import os 
import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
from num2words import num2words
from nltk.stem import WordNetLemmatizer
# Lemmatize with POS Tag
from nltk.corpus import wordnet
from num2words import num2words
from sklearn.metrics.pairwise import linear_kernel

title = "stories"
alpha = 0.3

## Preprocessing

### Lowercase

In [2]:
def convert_lower_case(data):
    return np.char.lower(data)

### Stopwords

In [3]:
# Stop words are the most commonly occurring words which don’t give any
# additional value to the document vector. in-fact removing these will
# increase computation and space efficiency. 

In [4]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
def remove_stop_words(data):
    new_text = ""
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))

    for word in words:
        if word not in stop_words and len(word) > 1:
            new_text = new_text + " " + word
        
    return new_text        

### Punctuation

In [6]:
# (Punctuation - dấu chấm câu)are the unnecessary symbols that are in
# our corpus documents, we should be little careful with what we are 
# doing with this. There might be few problems such as U.S — us 
# “United Stated” being converted to “us” after the preprocessing. 
# hyphen and should usually be dealt little carefully. but for this 
# problem statement we are just going to remove these

In [7]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [8]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

### Steamming

In [9]:
def stemming(data):
    new_text = ""
    words = word_tokenize(str(data))
    stemmer = PorterStemmer()
    
    for word in words:
        new_text = new_text + " " + stemmer.stem(word)
    return new_text

In [10]:
data = "The striped bats are hanging on their feet for best"
stemming(data)

' the stripe bat are hang on their feet for best'

### Lemmatizer words

#### Get word pos 

In [11]:
s = "i play so well"
nltk.pos_tag(s.split())

[('i', 'JJ'), ('play', 'VBP'), ('so', 'RB'), ('well', 'RB')]

In [12]:
s = "i play game"
nltk.pos_tag(word_tokenize(s))

[('i', 'JJ'), ('play', 'VBP'), ('game', 'NN')]

In [13]:
def get_wordnet_pos(word):

    tag = nltk.pos_tag([word])[0][1][0].upper()
    
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

#### Lemmatizer words

In [14]:
def lemmatizer_words(data):
    new_text = ""
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(str(data))
   
    lemmatizer_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in words]

    for word in lemmatizer_text:
        new_text = new_text + " " + word
    return  new_text

In [15]:
data = "The striped bats are hanging on their feet for best"
data
lemmatizer_words(data)

' The strip bat be hang on their foot for best'

### Convert Number 

In [16]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [17]:
data = "123"
convert_numbers(data)

array(' one hundred and twenty three', dtype='<U29')

### Combine all Preprocess step

In [18]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_stop_words(data)
    data = remove_apostrophe(data)
    data = convert_numbers(data)
    data = lemmatizer_words(data)
    data = stemming(data)
    
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = lemmatizer_words(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

## Extracting Data

In [19]:
links_small = pd.read_csv('./the-movies-dataset/links_small.csv')
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [20]:
links_small[:20]

0       862
1      8844
2     15602
3     31357
4     11862
5       949
6     11860
7     45325
8      9091
9       710
10     9087
11    12110
12    21032
13    10858
14     1408
15      524
16     4584
17        5
18     9273
19    11517
Name: tmdbId, dtype: int64

In [21]:
md = pd.read_csv('./the-movies-dataset/movies_metadata.csv')

/home/minhdeptrai/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
md = md.drop([19730,29503,35587])

In [23]:
md['id'] = md['id'].astype('int')

In [24]:
smd = md[md['id'].isin(links_small)]

In [25]:
N = len(smd)

In [26]:
smd[:20]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
5,False,NaN,60000000,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",...,1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0
6,False,NaN,58000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,...,1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0
7,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",...,1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0
8,False,NaN,35000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,...,1995-12-22,64350171.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Terror goes into overtime.,Sudden Death,False,5.5,174.0
9,False,"{'id': 645, 'name': 'James Bond Collection', '...",58000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://www.mgm.com/view/movie/757/Goldeneye/,710,tt0113189,en,GoldenEye,James Bond must unmask the mysterious head of ...,...,1995-11-16,352194034.0,130.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,No limits. No fears. No substitutes.,GoldenEye,False,6.6,1194.0


In [27]:
smd['tagline'] = smd['tagline'].fillna('')

/home/minhdeptrai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
smd['tagline'][:20]

0                                                      
1             Roll the dice and unleash the excitement!
2     Still Yelling. Still Fighting. Still Ready for...
3     Friends are the people who let you be yourself...
4     Just When His World Is Back To Normal... He's ...
5                              A Los Angeles Crime Saga
6     You are cordially invited to the most surprisi...
7                                The Original Bad Boys.
8                            Terror goes into overtime.
9                  No limits. No fears. No substitutes.
10    Why can't the most powerful man in the world h...
11                                                     
12                       Part Dog. Part Wolf. All Hero.
13    Triumphant in Victory, Bitter in Defeat. He Ch...
14    The Course Has Been Set. There Is No Turning B...
15                     No one stays at the top forever.
16             Lose your heart and come to your senses.
17    Twelve outrageous guests. Four scandalous 

In [29]:
smd['overview'] =smd['overview'].fillna('')

/home/minhdeptrai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
smd['description'] = smd['overview'] + smd['tagline']

/home/minhdeptrai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [31]:
smd['title'][:20]

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
5                               Heat
6                            Sabrina
7                       Tom and Huck
8                       Sudden Death
9                          GoldenEye
10            The American President
11       Dracula: Dead and Loving It
12                             Balto
13                             Nixon
14                  Cutthroat Island
15                            Casino
16             Sense and Sensibility
17                        Four Rooms
18    Ace Ventura: When Nature Calls
19                       Money Train
Name: title, dtype: object

### Word tokenize data 

In [32]:
processed_text = []
processed_title = []

for index, row in smd.iterrows():
    description = row['description']
    title = row['title']   
    processed_text.append(word_tokenize(str(preprocess(description))))
    processed_title.append(word_tokenize(str(preprocess(title))))

In [33]:
processed_text[:1]

[['lead',
  'woodi',
  'andi',
  'toy',
  'live',
  'happili',
  'room',
  'andi',
  'birthday',
  'bring',
  'buzz',
  'lightyear',
  'onto',
  'scene',
  'afraid',
  'lose',
  'place',
  'andi',
  'heart',
  'woodi',
  'plot',
  'buzz',
  'circumst',
  'separ',
  'buzz',
  'woodi',
  'owner',
  'duo',
  'eventu',
  'learn',
  'put',
  'asid',
  'differ']]

In [34]:
processed_title[:10]

[['toy', 'stori'],
 ['jumanji'],
 ['grumpier', 'old', 'men'],
 ['wait', 'exhal'],
 ['father', 'bride', 'part', 'ii'],
 ['heat'],
 ['sabrina'],
 ['tom', 'huck'],
 ['sudden', 'death'],
 ['goldeney']]

## Create Corpus Bag 

In [35]:
# Mục đích là để tạo ra một bộ từ điển liệt kê tất cả các các từ và danh sách các bộ phim có sự xuất hiện của nó 


DF = {}

for i in range(N):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i} #Trừ khi trong DF[w] có tồn tại i
    
    tokens = processed_title[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}


In [36]:
# tính tần xuất xuất hiện của từ đó trên toàn bộ danh sách bộ phim 
for i in DF:
    DF[i] = len(DF[i])

In [37]:
total_vocab_size = len(DF)
total_vocab_size

22406

In [38]:
total_vocab = [x for x in DF]
total_vocab[:10]

['lead',
 'woodi',
 'andi',
 'toy',
 'live',
 'happili',
 'room',
 'birthday',
 'bring',
 'buzz']

In [39]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

## Calcualte DF-IDF

### Calcalate DF-IDF from description

In [40]:
doc  = 0
tf_idf = {}

for i in range(N):
    
    tokens = processed_text[i]
    
    # counting frequence the number of word in a document
    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N)/(df + 1))
        
        tf_idf[doc, token] = tf*idf
    
    doc += 1  

### Calcalate DF-IDF from title

In [41]:
tf_idf_title = {}
doc = 0

for i in range(N):
    
    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len (tokens + processed_text[i])
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log(N/(df + 1))
        
        tf_idf_title[doc, token] = tf*idf
    doc += 1 

In [42]:
tf_idf_title[(0,"toy")]

0.3325761674487064

In [43]:
tf_idf[(0,"toy")]

0.3325761674487064

### Merging the TF-IDF according to weights

In [44]:
#TF-IDF(document) = TF-IDF(body) * alpha + TF-IDF(title) * (1-alpha)

In [45]:
tf_idf_text = tf_idf 

for i in tf_idf:
    tf_idf[i] *= alpha

for i in tf_idf_title:
    tf_idf_title[i] *= (1 - alpha)

In [46]:
# shitty code
# thực ra nó lặp trên tf_idf nếu thằng nào mà có mặt trên cả title và body thì if_idf của nó tính bằng body và
# title đều như nhau.
# nếu 1 thằng word nào đó mà không có mặt trên tf_idf_title thì tf_idf_title[i] không tồn tại => safe, nó sẽ có 
# giá trị bằng tf_idf trên body nhân với alpha

In [47]:
for i in tf_idf_title:
    if i in tf_idf: tf_idf[i] = tf_idf[i] + tf_idf_title[i]
    else : tf_idf[i] = tf_idf_title[i]

tf_idf[(0,"toy")]

0.33257616744870633

### Mathching score method 

In [48]:
# ý tưởng của mathching score là tìm tất cả các ký tự giống nhau giữa từng đoạn văn bản và query rồi tính tổng
# giá trị của các ký tự đó (đối với mỗi đoạn văn bản một) rồi lập list xếp theo giá trị query_weights

def matching_score(query):
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("Matching score")
    print("\n Query:", query)
    print("")
    print(tokens)
    
    query_weights = {}
    
    for key in tf_idf:
        if key[1] in tokens:
            try: 
                query_weights[key[0]] += tf_idf[key]
            except:
                query_weights[key[0]] = tf_idf[key]
    
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse =True)
    
    
    print("")
    
    l = []
    
    for i in query_weights[:10]:
         print(smd.iloc[i[0]]['title']) 

In [49]:
matching_score("The Dark Knight")

Matching score

 Query: The Dark Knight

['dark', 'knight']

Last Knights
Dark Water
The Seeker: The Dark Is Rising
The Dark Knight Rises
The Hollywood Knights
A Knight's Tale
Tales from the Crypt: Demon Knight
Thor: The Dark World
A Shot in the Dark
Dark Habits


In [50]:
smd[smd['title'] == "The Dark Knight"]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,description
12481,False,"{'id': 263, 'name': 'The Dark Knight Collectio...",185000000,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",http://thedarkknight.warnerbros.com/dvdsite/,155,tt0468569,en,The Dark Knight,Batman raises the stakes in his war on crime. ...,...,1.004558e+09,152.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Why So Serious?,The Dark Knight,False,8.3,12269.0,Batman raises the stakes in his war on crime. ...


### TF Cosine Similarity Ranking

In [51]:
# hàm tính đố tương đồng giữa 2 vector 
def cosine_sim(a,b):
    cos_sim = np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

#### Vectorizing tf-idf

In [52]:
# lập danh sách các vector 
# ma trận D ban đầu có giá trị toàn bằng 0, sau đó chúng ta sẽ duyệt tf_idf và trám các giá trị có tồn tại vào D
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1]) # vị trí của từ trong vocabolary 
        D[i[0]][ind] = tf_idf[i] 
    except:
            pass

In [53]:
D

array([[0.0247831 , 0.15612165, 0.14610684, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [54]:
# tạo vector từ query, nếu từ nào không xuất hiện thì tần suất xuất hiện của nó bằng 0
def gen_vector(tokens):
    
    Q = np.zeros(total_vocab_size)
    
    counter = Counter(tokens)
    words_count = len(tokens)
    
    
    for token in np.unique(tokens):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log(N/(df + 1))
        
        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    
    return Q

In [55]:
#query = "Dark night"

#preprocessed_query = preprocess(query)
#tokens = word_tokenize(str(preprocessed_query))
#array =  gen_vector(tokens)

#for i in array: 
#    print(i)

In [56]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))
    
    print("\n Query:", query)
    print("")
    print(tokens)
    
    d_cosines = []
    
    query_vector = gen_vector(tokens)
    
    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))
    
    out = np.array(d_cosines).argsort()[-k:][::-1]
    
    print("")
    
    for i in out:
       print(smd.iloc[i]['title']) 
    
    

In [57]:
Q = cosine_similarity(10, "The Dark Knight")

Cosine Similarity

 Query: The Dark Knight

['dark', 'knight']

The Dark Knight Rises
Last Knights
The Hollywood Knights
A Knight's Tale
Dark Water
The Dark Knight
The Seeker: The Dark Is Rising
Black Knight
Knight and Day
Dark Water


In [58]:
Q = cosine_similarity(10, "The Godfather")

Cosine Similarity

 Query: The Godfather

['godfath']

The Godfather: Part III
Tokyo Godfathers
The Godfather
The Godfather: Part II
Bright Eyes
Things to Do in Denver When You're Dead
Free Enterprise
The Freshman
Jane Austen's Mafia!
Cinderfella


In [59]:
Q = cosine_similarity(10, "wanted")

Cosine Similarity

 Query: wanted

['want']

All I Want For Christmas
Wanted
A Most Wanted Man
Madagascar 3: Europe's Most Wanted
Wilbur Wants to Kill Himself
What Women Want
The Interview
The Pelican Brief
Music Box
Everything You Always Wanted to Know About Sex *But Were Afraid to Ask


In [60]:
cosine_sim = linear_kernel(D, D)

In [61]:
cosine_sim[0]

array([0.22474677, 0.0016116 , 0.        , ..., 0.        , 0.00033573,
       0.        ])

In [62]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [63]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [64]:
get_recommendations('The Godfather').head(10)

973         The Godfather: Part II
7760                 Henry's Crime
1582       The Godfather: Part III
9046    Michael Jackson's Thriller
9017        Manson Family Vacation
3031               The Five Senses
5461              Tokyo Godfathers
1871                 Godzilla 1985
3531          Camille Claudel 1915
8574          Camille Claudel 1915
Name: title, dtype: object

In [65]:
get_recommendations('The Dark Knight').head(10)

6900                            The Dark Knight
1113                             Batman Returns
7565                 Batman: Under the Red Hood
7931                      The Dark Knight Rises
8840                               Last Knights
8165    Batman: The Dark Knight Returns, Part 1
2579               Batman: Mask of the Phantasm
6144                              Batman Begins
524                                      Batman
132                              Batman Forever
Name: title, dtype: object